In [1]:
#setting up the arena without orientation consideration

In [2]:
import string
from itertools import permutations

In [3]:
def get_x_coordinate(coordinate): #get int x-coordinate from string coordinate
    x_coordinate = string.ascii_uppercase.index(coordinate[0])
    return x_coordinate

def get_y_coordinate(coordinate): #get int y-coordinate from string coordinate
    y_coordinate = int(coordinate[2:])
    return y_coordinate

def get_alphanumeric_coordinates(x, y): #convert to alphanumeric coordinates
    alphanumeric_coordinate = '{alphabet}-{number}'.format(
        alphabet=string.ascii_uppercase[x],
        number=y)
    return alphanumeric_coordinate

In [4]:
def get_coordinates():
    coordinates = {}
    for i in range(20):
        for j in range(20):
            coordinates[get_alphanumeric_coordinates(i, j)] = (i, j)
    return coordinates

In [5]:
def get_default_graph():
    
    graph = {}
    DISTANCE = 10
    
    for i in range(20):
        for j in range(20):
            adjacent_coordinates = []
            if i!=0: #if not left edge, add left adjacent to list
                adajcent_coordinate = get_alphanumeric_coordinates(i-1, j)
                adjacent_coordinates.append((adajcent_coordinate, DISTANCE))
            if i!=19: #if not right edge, add right adjacent to list
                adajcent_coordinate = get_alphanumeric_coordinates(i+1, j)
                adjacent_coordinates.append((adajcent_coordinate, DISTANCE))
            if j!=0: #if not bottom edge, add bottom adjacent to list
                adajcent_coordinate = get_alphanumeric_coordinates(i, j-1)
                adjacent_coordinates.append((adajcent_coordinate, DISTANCE))
            if j!=19: #if not top edge, add top adjacent to list
                adajcent_coordinate = get_alphanumeric_coordinates(i, j+1)
                adjacent_coordinates.append((adajcent_coordinate, DISTANCE))
            
            graph[get_alphanumeric_coordinates(i, j)] = adjacent_coordinates
            
    return graph

In [6]:
def get_virtual_boundary(obstacles): #add virtual boundary of 50x50cm surrounding each obstacle

    boundary = [] #nodes to remove from graph
    
    for obstacle in obstacles:

        obstacle_coordinate = obstacle[0]
        x_coordinate = get_x_coordinate(obstacle_coordinate)
        y_coordinate = get_y_coordinate(obstacle_coordinate)

        left_bounds = x_coordinate - 2
        right_bounds = x_coordinate + 2
        top_bounds = y_coordinate + 2
        bottom_bounds = y_coordinate - 2
        
        for i in range(left_bounds, right_bounds+1):
            for j in range(bottom_bounds, top_bounds+1):
                if i>=0 and i<=19 and j>=0 and j<=19:
                    coordinate_to_remove = string.ascii_uppercase[i] + "-" + str(j)
                    boundary.append(coordinate_to_remove)
        
    return boundary

In [7]:
def get_goals(obstacles):
    goals = []
    
    #assume goal is node 30cm away from obstacle in the opposite direction
    for obstacle in obstacles:
        
        direction = obstacle[1]
        obstacle_x = get_x_coordinate(obstacle[0])
        obstacle_y = get_y_coordinate(obstacle[0])
        
        if direction == "N": #goal is 30cm above obstacle
            goal_x = obstacle_x
            goal_y = obstacle_y + 3
        elif direction == "S": #goal is 30cm below obstacle
            goal_x = obstacle_x
            goal_y = obstacle_y - 3
        elif direction == "E": #goal is 30cm right of obstacle
            goal_x = obstacle_x + 3
            goal_y = obstacle_y
        elif direction == "W": #goal is 30cm right of obstacle
            goal_x = obstacle_x - 3
            goal_y = obstacle_y
            
        goal = get_alphanumeric_coordinates(goal_x, goal_y)
        goals.append(goal)
        
    return goals

In [8]:
def create_graph(obstacles):
    default_graph = get_default_graph()
    boundaries = get_virtual_boundary(obstacles)
    
    for boundary in boundaries: #remove boundary nodes from graph keys
        default_graph.pop(boundary, None)
        
    for node, adjacent_nodes in default_graph.items(): #remove boundary nodes from adjacency list
        for boundary in boundaries:
            for adjacent_node in adjacent_nodes:
                if boundary == adjacent_node[0]:
                    adjacent_nodes.remove(adjacent_node)
        default_graph[node] = adjacent_nodes
        
    return default_graph

In [9]:
def permutate_goals(obstacles):
    goals = get_goals(obstacles)
    permutated_goals = permutations(goals)
    return [list(p) for p in permutated_goals]